<a href="https://colab.research.google.com/github/midnightripper/accuracy_improvement/blob/main/VAE_(19%2C16%2C12%2C8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Prerequisite

In [2]:
pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import scipy.io as sio
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Input
from keras.layers import Reshape
from tensorflow.keras import backend as K

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Load the train and test data splits

In [4]:
GER_test = (np.array(sio.loadmat('/content/drive/MyDrive/training data/GER_test.mat')['GER_test'])).transpose()
GER_train = (np.array(sio.loadmat('/content/drive/MyDrive/training data/GER_train.mat')['GER_train'])).transpose()
test_frame = pd.DataFrame(GER_test).sort_values(1)
train_frame = pd.DataFrame(GER_train).sort_values(1)

In [5]:
print(f"x_train shape: {GER_train.shape} - y_train shape: {GER_train.shape}")
print(f"x_test shape: {GER_test.shape} - y_test shape: {GER_test.shape}")

x_train shape: (6981, 21) - y_train shape: (6981, 21)
x_test shape: (6248, 21) - y_test shape: (6248, 21)


In [6]:
def get_data_labels(split):
    x = pd.DataFrame(split)
    labels = x[0].values.astype(np.uint8)
    del x[0],x[1]
    data = x.values 
    return data, labels
    
x_train, y_train = get_data_labels(train_frame.values.tolist())
x_test, y_test = get_data_labels(test_frame.values.tolist())

In [7]:
df = pd.DataFrame(x_train)
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [8]:
df2 = pd.DataFrame(x_test)
scaler = StandardScaler()
x_test = pd.DataFrame(scaler.fit_transform(df2), columns=df.columns)

In [9]:
print(x_train.mean())

0     5.699813e-17
1    -1.628518e-17
2     1.791370e-16
3     8.142590e-18
4    -4.071295e-18
5    -4.274860e-16
6    -8.676947e-17
7     8.549719e-17
8     7.302885e-17
9    -4.885554e-17
10    9.771108e-17
11    3.257036e-17
12    1.302814e-16
13    8.386867e-16
14    1.526736e-18
15    2.157786e-16
16    2.015291e-16
17    9.440315e-17
18    8.193481e-17
dtype: float64


In [10]:
print(x_train.std())

0     1.000072
1     1.000072
2     1.000072
3     1.000072
4     1.000072
5     1.000072
6     1.000072
7     1.000072
8     1.000072
9     1.000072
10    1.000072
11    1.000072
12    1.000072
13    1.000072
14    1.000072
15    1.000072
16    1.000072
17    1.000072
18    1.000072
dtype: float64


In [11]:
print(f"Size of the training set: {len(x_train)}")

Size of the training set: 6981


In [12]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print(f"Size of the x training set: {len(x_train)}")
print(f"Size of the x validation set: {len(x_val)}")
print(f"Size of the y training set: {len(y_train)}")
print(f"Size of the y validation set: {len(y_val)}")

Size of the x training set: 5584
Size of the x validation set: 1397
Size of the y training set: 5584
Size of the y validation set: 1397


In [13]:
print(x_train[:5])
print(y_train[:5])

            0         1         2         3         4         5         6   \
6565 -1.076680 -1.065748 -1.012843 -0.799906 -0.713923 -0.710367 -1.932554   
4003  0.582947  1.135969  1.297658  1.015694  0.868769 -0.130759 -1.806106   
1010 -0.545586 -0.653364 -0.430347 -0.782925 -0.775639 -0.323475 -0.652264   
4617 -0.566326 -0.578170 -0.686170 -0.379677 -0.210357 -1.287966  0.392749   
931   0.245015  0.061639  0.443537 -0.616326 -0.935091  0.915754  0.721268   

            7         8         9         10        11        12        13  \
6565  2.396384 -0.985383 -0.853229 -0.838086 -0.790418 -0.770441  1.082992   
4003  1.601756 -0.026166  1.974222  2.026149  2.399590  2.084844 -0.682288   
1010  0.034567 -1.103360  0.015721  0.285905  0.997402  0.964844 -2.012156   
4617 -0.268374 -0.030353 -0.463698 -0.261271  1.170901  1.321986 -2.641625   
931  -0.828500 -0.803408 -0.062883 -0.065124 -0.179201 -0.313890  0.995570   

            14        15        16        17        18  
6565 

Early stopping to prevent over-fitting

In [14]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)





# Function Calls

In [15]:
input_shape = (19,)

In [16]:
# Define the sampling layer
# def sampling(args):
#     z_mean, z_log_var = args
#     epsilon = tf.keras.backend.random_normal(shape=(tf.keras.backend.shape(z_mean)[0], 19), mean=0., stddev=0.00001)
#     return z_mean + tf.keras.backend.exp(0.5 * z_log_var) * epsilon

def sampling(args):
    """Reparameterization trick. Instead of sampling from Q(z|X), 
    sample eps = N(0,I) z = z_mean + sqrt(var)*eps.

    Parameters:
    -----------
    args: list of Tensors
        Mean and log of variance of Q(z|X)

    Returns
    -------
    z: Tensor
        Sampled latent vector
    """

    z_mean, z_log_var = args
    eps = tf.random.normal(tf.shape(z_log_var), dtype=tf.float32, mean=0., stddev=1 , name='epsilon')
    z = z_mean + tf.exp(z_log_var / 2) * eps
    return z

# Encoder

In [17]:
inputs = keras.layers.Input(shape=input_shape, name='input')
x = keras.layers.Dense(36, activation='relu')(inputs)
# x = layers.Dropout(0.5)(x)
x = keras.layers.Dense(72, activation='relu')(x)
# x = layers.Dropout(0.5)(x)
z_mean = keras.layers.Dense(36, name='z_mean')(x)
z_log_var = keras.layers.Dense(36, name='z_log_var')(x)

In [18]:
z = keras.layers.Lambda(sampling, name='z')([z_mean, z_log_var])
encoder = keras.Model(inputs, z , name='encoder')
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 19)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 36)           720         ['input[0][0]']                  
                                                                                                  
 dense_1 (Dense)                (None, 72)           2664        ['dense[0][0]']                  
                                                                                                  
 z_mean (Dense)                 (None, 36)           2628        ['dense_1[0][0]']                
                                                                                            

# Decoder

In [19]:
#Giving input to decoder and initialising shape
# latent_inputs = keras.layers.Input(shape=(8), name='z_sampling')

latent_inputs = keras.layers.Dense(72, activation='relu')(z)
# x = layers.Dropout(0.5)(latent_inputs)
x = keras.layers.Dense(36, activation='relu')(latent_inputs)
# x = layers.Dropout(0.5)(x)
# middle=keras.layers.Dense(16, activation='relu')(x)
outputs = keras.layers.Dense(19, activation='relu')(x)


# instantiate decoder model
decoder = keras.Model(latent_inputs, outputs, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 72)]              0         
                                                                 
 dense_3 (Dense)             (None, 36)                2628      
                                                                 
 dense_4 (Dense)             (None, 19)                703       
                                                                 
Total params: 3,331
Trainable params: 3,331
Non-trainable params: 0
_________________________________________________________________


In [20]:
print(outputs)

KerasTensor(type_spec=TensorSpec(shape=(None, 19), dtype=tf.float32, name=None), name='dense_4/Relu:0', description="created by layer 'dense_4'")


# Implementing VAE without Loss

# Initialising Loss 

In [21]:
autoencoder = Model(inputs, outputs)

In [22]:
autoencoder.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 19)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 36)           720         ['input[0][0]']                  
                                                                                                  
 dense_1 (Dense)                (None, 72)           2664        ['dense[0][0]']                  
                                                                                                  
 z_mean (Dense)                 (None, 36)           2628        ['dense_1[0][0]']                
                                                                                              

In [23]:
reconstruction_loss = K.mean(K.square(outputs - inputs), axis=-1)
kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
vae_loss = K.mean(reconstruction_loss + kl_loss)

# Training VAE

In [24]:
autoencoder.add_loss(vae_loss)
autoencoder.summary()
autoencoder.compile(optimizer='adam')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 19)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 36)           720         ['input[0][0]']                  
                                                                                                  
 dense_1 (Dense)                (None, 72)           2664        ['dense[0][0]']                  
                                                                                                  
 z_mean (Dense)                 (None, 36)           2628        ['dense_1[0][0]']                
                                                                                              

In [25]:
history = autoencoder.fit(x_train, x_train,
                epochs=200,
                batch_size=128,
                shuffle=True,
                validation_data=(x_val,x_val))

Epoch 1/200
44/44 [==============================] - 3s 14ms/step - loss: 1.9324 - val_loss: 1.2764
Epoch 2/200
44/44 [==============================] - 0s 6ms/step - loss: 1.1479 - val_loss: 1.1271
Epoch 3/200
44/44 [==============================] - 0s 8ms/step - loss: 1.0629 - val_loss: 1.0764
Epoch 4/200
44/44 [==============================] - 0s 9ms/step - loss: 1.0274 - val_loss: 1.0520
Epoch 5/200
44/44 [==============================] - 0s 8ms/step - loss: 1.0104 - val_loss: 1.0413
Epoch 6/200
44/44 [==============================] - 0s 6ms/step - loss: 1.0020 - val_loss: 1.0361
Epoch 7/200
44/44 [==============================] - 0s 7ms/step - loss: 1.0003 - val_loss: 1.0328
Epoch 8/200
44/44 [==============================] - 0s 6ms/step - loss: 0.9967 - val_loss: 1.0313
Epoch 9/200
44/44 [==============================] - 0s 6ms/step - loss: 0.9962 - val_loss: 1.0302
Epoch 10/200
44/44 [==============================] - 0s 6ms/step - loss: 0.9970 - val_loss: 1.0298
Epoch 11

# Extracting the Encoder part of VAE and freezing them

In [26]:
encoder = Model(inputs, z)
encoder.trainable = False
encoder.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 19)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 36)           720         ['input[0][0]']                  
                                                                                                  
 dense_1 (Dense)                (None, 72)           2664        ['dense[0][0]']                  
                                                                                                  
 z_mean (Dense)                 (None, 36)           2628        ['dense_1[0][0]']                
                                                                                            

In [27]:
classifier_input = Input(shape=(19,))
encoded_input = encoder(classifier_input)
x1=Dense(128,activation='relu')(encoded_input)
x2=layers.Dropout(0.5)(x1)
x2=Dense(64,activation='relu')(x2)
x2=layers.Dropout(0.25)(x2)
x3=Dense(32,activation='relu')(x2)
x4=Dense(16,activation='relu')(x3)
x5=Dense(8,activation='relu')(x4)
x6=Dense(4,activation='relu')(x5)
output_layer = Dense(1, activation='sigmoid')(x6)

classifier = Model(classifier_input, output_layer)
classifier.summary()
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_val, y_val),callbacks=[es])


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 19)]              0         
                                                                 
 model_1 (Functional)        (None, 36)                8640      
                                                                 
 dense_5 (Dense)             (None, 128)               4736      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 32)                2080

In [28]:
accuracy = classifier.evaluate(x_train, y_train)[1]
print(f"Train accuracy: {round(accuracy * 100, 2)}%")

175/175 [==============================] - 0s 2ms/step - loss: 0.6856 - accuracy: 0.5614
Train accuracy: 56.14%


In [29]:
accuracy = classifier.evaluate(x_test, y_test)[1]
print(f"Test accuracy: {round(accuracy * 100, 2)}%")

196/196 [==============================] - 0s 2ms/step - loss: 0.6862 - accuracy: 0.5589
Test accuracy: 55.89%


In [30]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from keras.layers import Input, Dense
from keras.models import Model
import seaborn as sns

In [31]:
import seaborn as sns

In [ ]:
encoded_data = autoencoder.predict(x_train)
print(encoded_data.shape)
tsne = TSNE(n_components=2, verbose=1, random_state=123)
z = tsne.fit_transform(encoded_data)
df = pd.DataFrame()
df["y"] = y_train
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]

sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(),
                palette=sns.color_palette("hls", 10),
                data=df).set(title="MNIST data T-SNE projection")

175/175 [==============================] - 0s 1ms/step
(5584, 19)
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 5584 samples in 0.001s...
[t-SNE] Computed neighbors for 5584 samples in 0.327s...
[t-SNE] Computed conditional probabilities for sample 1000 / 5584
[t-SNE] Computed conditional probabilities for sample 2000 / 5584
[t-SNE] Computed conditional probabilities for sample 3000 / 5584
[t-SNE] Computed conditional probabilities for sample 4000 / 5584
[t-SNE] Computed conditional probabilities for sample 5000 / 5584

In [ ]:
encoded_data = encoder.predict(x_train)
print(encoded_data.shape)
tsne = TSNE(n_components=2, verbose=1, random_state=123)
z = tsne.fit_transform(encoded_data)
df = pd.DataFrame()
df["y"] = y_train
df["comp-1"] = z[:,0]
df["comp-2"] = z[:,1]

sns.scatterplot(x="comp-1", y="comp-2", hue=df.y.tolist(),
                palette=sns.color_palette("hls", 10),
                data=df).set(title="MNIST data T-SNE projection")

# Measuring Accuracy